In [132]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import math

# Obter lista de bares por cidade

In [204]:
lista_bares = []

mais_paginas = True
url = 'https://comidadibuteco.com.br/category/butecos/montes-claros/'
response = urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
total_bares = re.findall(r'\d+', soup.h5.getText())
total_bares = int(total_bares[0])
total_paginas = math.ceil(total_bares / 12)

In [205]:
i = 0
lista_bares = []

while i < total_paginas:
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    lista_bares.extend(soup.findAll('div', {'class': 'result-inner text-left'}))
    i += 1
    if i < total_paginas:
        url = soup.find('div', {'class': 'navigation'}).find('div', {'class': 'alignright'}).a['href']

# Criar dataframe de bares

In [206]:
import pandas as pd
import re

In [207]:
bar_detalhes = []

for bar in lista_bares:
    bar_atual = {}
    bar_atual['nome'] = (bar.h2.getText())
    endereco = bar.p.getText()
    endereco_limpo = re.sub(r'\s+', ' ', endereco.strip()).replace(' |', ',')
    bar_atual['endereco'] = endereco_limpo
    
    bar_detalhes.append(bar_atual)

In [208]:
bar_df = pd.DataFrame(bar_detalhes)

In [209]:
bar_df.head()

,nome,endereco
0,611 House Bar,"Rua Raul Correia, 611, Funcionários, Montes Cl..."
1,Ary Cristal,"Rua Raul Correia, 610, Funcionários, Montes Cl..."
2,Bar do Chacal,"Av. Sidney Chaves, 779, Edgar Pereira, Montes ..."
3,Bar do Kal,"Rua Antônio Martins, 50, Santos Reis, Montes C..."
4,Bar do Soró,"Rua Coronel Joaquim Sarmento, 596, São José, M..."


# Obtendo latitude e longitude

In [210]:
import requests
import time
import folium
from geopy import Point, distance
from geopy.geocoders import Nominatim
from api_key import API_KEY

In [231]:
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'address': endereco, 'key': API_KEY}

resposta = requests.get(url, params=params).json()

latitude = resposta['results'][0]['geometry']['location']['lat']
longitude = resposta['results'][0]['geometry']['location']['lng']

print(latitude, longitude)

-16.7290615 -43.8741313


In [232]:
def obter_lat_long(endereco):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': endereco, 'key': API_KEY}
    resposta = requests.get(url, params=params).json()
    try:
        lat = resposta['results'][0]['geometry']['location']['lat']
        long = resposta['results'][0]['geometry']['location']['lng']
        return pd.Series({'latitude': lat, 'longitude': long})
    except:
        return pd.Series({'latitude': None, 'longitude': None})

In [233]:
bar_df[['latitude', 'longitude']] = bar_df['endereco'].apply(obter_lat_long)
bar_df.dropna(inplace=True)
bar_df.head()

,nome,endereco,latitude,longitude
0,611 House Bar,"Rua Raul Correia, 611, Funcionários, Montes Cl...",-16.738308,-43.877505
1,Ary Cristal,"Rua Raul Correia, 610, Funcionários, Montes Cl...",-16.738293,-43.877752
2,Bar do Chacal,"Av. Sidney Chaves, 779, Edgar Pereira, Montes ...",-16.710061,-43.859608
3,Bar do Kal,"Rua Antônio Martins, 50, Santos Reis, Montes C...",-16.701821,-43.870259
4,Bar do Soró,"Rua Coronel Joaquim Sarmento, 596, São José, M...",-16.722578,-43.856029


# Representando localização dos bares no mapa

In [234]:
centroide = Point(sum(bar_df['latitude'])/len(bar_df), sum(bar_df['longitude'])/len(bar_df))

ponto_mais_distante = None
distancia_mais_longe = 0
for i, row in bar_df.iterrows():
    p = Point(row['latitude'], row['longitude'])
    dist = distance.distance(p, centroide).km
    if dist > distancia_mais_longe:
        distancia_mais_longe = dist
        ponto_mais_distante = p

map = folium.Map(location=[ponto_mais_distante.latitude, ponto_mais_distante.longitude], zoom_start=12)
for i, row in bar_df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], tooltip=row['nome']).add_to(map)
map

# Calculando ditância entre bares

In [235]:
from geopy import distance

In [238]:
bar_df[['proximos_5', 'proximos_2']] = bar_df.apply(lambda x: pd.Series(encontrar_proximos(x, bar_df)), axis=1)

ValueError: Columns must be same length as key

In [239]:
bar_df

,nome,endereco,latitude,longitude,distancia
0,611 House Bar,"Rua Raul Correia, 611, Funcionários, Montes Cl...",-16.738308,-43.877505,1.084675
1,Ary Cristal,"Rua Raul Correia, 610, Funcionários, Montes Cl...",-16.738293,-43.877752,1.092116
2,Bar do Chacal,"Av. Sidney Chaves, 779, Edgar Pereira, Montes ...",-16.710061,-43.859608,2.611553
3,Bar do Kal,"Rua Antônio Martins, 50, Santos Reis, Montes C...",-16.701821,-43.870259,3.042709
4,Bar do Soró,"Rua Coronel Joaquim Sarmento, 596, São José, M...",-16.722578,-43.856029,2.059408
5,Bar do Thom,"Rua Humaitá, 50, Morrinhos, Montes Claros - MG",-16.735713,-43.861069,1.575482
6,Bendita Espeteria,"Avenida Cula Mangabeira, 661, Santo Expedito, ...",-16.734650,-43.872087,0.655751
7,Boteco Topp,Avenida Cula Mangabeira - de 564 ao fim - lado...,-16.736361,-43.872104,0.836241
8,Butiquim do Loy,"Rua Crisântemo, 712, Sagrada Família, Montes C...",-16.739607,-43.879511,1.300414
9,Cantina da Paty,"Alameda A, 200, Acácias, Montes Claros MG",-16.723965,-43.819757,5.825774
